In [2]:
import pandas as pd
import numpy as np

In [3]:
def short_rate_tree(n,init_rate,u,d):
    tree = np.zeros((n+1,n+1))
    for i in range(n+1):
        for j in range(i+1):
            tree[i][j]=r00*u**j*d**(i-j)
    rate_df = pd.DataFrame(data=tree, columns=np.arange(tree.shape[0]))        
    return rate_df

In [4]:
r00 = 0.05
u = 1.1
d = 0.9
q = 1/2
n = 10
short_df = short_rate_tree(n=n, init_rate=r00, u=u, d=d)
short_df = short_df * 100
short_df

,0,1,2,3,4,5,6,7,8,9,10
0,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,4.500000,5.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,4.050000,4.950000,6.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3.645000,4.455000,5.445000,6.655000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,3.280500,4.009500,4.900500,5.989500,7.320500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2.952450,3.608550,4.410450,5.390550,6.588450,8.052550,0.000000,0.000000,0.000000,0.000000,0.000000
6,2.657205,3.247695,3.969405,4.851495,5.929605,7.247295,8.857805,0.000000,0.000000,0.000000,0.000000
7,2.391485,2.922926,3.572465,4.366346,5.336645,6.522566,7.972025,9.743586,0.000000,0.000000,0.000000
8,2.152336,2.630633,3.215218,3.929711,4.802980,5.870309,7.174822,8.769227,10.717944,0.000000,0.000000
9,1.937102,2.367570,2.893696,3.536740,4.322682,5.283278,6.457340,7.892304,9.646150,11.789738,0.000000


In [5]:
def zero_coupon_bond(short_df, face_value, maturity,q):
    price_tree = np.ones((maturity+1,maturity+1))*face_value
    rate_tree = short_df.to_numpy()
    for i in range(maturity-1,-1,-1):
        for j in range(i+1):
            price_tree[i][j]=1/(1+(rate_tree[i][j]/100))*(q*price_tree[i+1][j+1]+(1-q)*price_tree[i+1][j])
    
    price_tree[np.triu_indices(price_tree.shape[0], 1)] = 0
    price_df = pd.DataFrame(data=price_tree, columns=np.arange(price_tree.shape[0]))
    return price_df

def forward_price(short_df, maturity, q=0.5):
    n=short_df.shape[0]-1
    mat_zcb_df = zero_coupon_bond(short_df=short_df, face_value=1, maturity=maturity, q=q)
    mat_zcb_price = mat_zcb_df.iloc[0,0]
    n_zcb_df = zero_coupon_bond(short_df=short_df, face_value=100, maturity=n, q=q).iloc[:maturity+1, :maturity+1]
    rate_tree = short_df.to_numpy()
    price_tree = n_zcb_df.to_numpy()
    for i in range(maturity-1,-1,-1):
        for j in range(i+1):
            price_tree[i][j]=1/(1+(rate_tree[i][j]/100))*(q*price_tree[i+1][j+1]+(1-q)*price_tree[i+1][j])
            
    price_df = np.round(pd.DataFrame(data=price_tree, columns=np.arange(price_tree.shape[0])), 4) 
    forward_price = price_df.iloc[0,0] / mat_zcb_price
    return mat_zcb_price, forward_price, price_df

def future_price(short_df, maturity, q=0.5):
    n=short_df.shape[0]-1
    n_zcb_df = zero_coupon_bond(short_df=short_df, face_value=100, maturity=n, q=q).iloc[:maturity+1, :maturity+1]
    rate_tree = short_df.to_numpy()
    price_tree = n_zcb_df.to_numpy()
    for i in range(maturity-1,-1,-1):
        for j in range(i+1):
            price_tree[i][j]=(q*price_tree[i+1][j+1]+(1-q)*price_tree[i+1][j])
            
    price_df = np.round(pd.DataFrame(data=price_tree, columns=np.arange(price_tree.shape[0])), 4) 
    future_price = price_df.iloc[0,0] 
    return future_price, price_df


### Question 1

Background: please start by building an n=10-period binomial model for the short-rate, $r_{i,j}$ . The lattice parameters are: $r_{0,0} = 5\%$, $u = 1.1$, $d = 0.9$ and $q = 1-q = \frac{1}{2}$
Compute the price of a zero-coupon bond (ZCB) that matures at time t=10 and that has face value 100.

Submission Guideline: Give your answer rounded to 2 decimal places. For example, if you compute the answer to be 73.2367%, submit 73.24.

In [6]:
zcb_10_df = zero_coupon_bond(short_df=short_df, face_value=100, maturity=n, q=0.5)   
print(f"The price of 10 Year zero coupon Bond is {np.round(zcb_10_df.iloc[0,0], 2)}")
zcb_10_df.iloc[:5,:5]

The price of 10 Year zero coupon Bond is 61.62


,0,1,2,3,4
0,61.621958,0.000000,0.000000,0.000000,0.000000
1,67.441030,61.965082,0.000000,0.000000,0.000000
2,72.881830,68.069922,62.676402,0.000000,0.000000
3,77.886862,73.780227,69.098538,63.838111,0.000000
4,82.422216,79.029459,75.104814,70.617093,65.555982


### Question 2 

Background: Please use the same binomial model as the previous question.

Compute the price of a forward contract on the same ZCB of the previous question where the forward contract matures at time t=4. 

Submission Guideline: Give your answer rounded to 2 decimal places. For example, if you compute the answer to be 73.2367%, submit 73.24.

In [7]:
mzp, fp, forward_df = forward_price(short_df=short_df, maturity=4)
print(f"The 4 year ZCB is {np.round(mzp, 2)} and the Forward Price is {np.round(fp, 2)}")
forward_df


The 4 year ZCB is 0.82 and the Forward Price is 74.88


,0,1,2,3,4
0,61.6220,0.0000,0.0000,0.0000,0.000
1,67.4410,61.9651,0.0000,0.0000,0.000
2,72.8818,68.0699,62.6764,0.0000,0.000
3,77.8869,73.7802,69.0985,63.8381,0.000
4,82.4222,79.0295,75.1048,70.6171,65.556


### Question 3

Background: Please use the same binomial model as the previous question.

Compute the initial price of a futures contract on the same ZCB of the previous two questions. The futures contract has an expiration of t=4.

Submission Guideline: Give your answer rounded to 2 decimal places. For example, if you compute the answer to be 73.2367%, submit 73.24.


In [8]:
fut_price, future_df = future_price(short_df=short_df, maturity=4)
print(f"The Future's Price with 4 year maturity is {np.round(fut_price, 2)}")
future_df

The Future's Price with 4 year maturity is 74.82


,0,1,2,3,4
0,74.8246,0.0000,0.0000,0.0000,0.000
1,76.9303,72.7189,0.0000,0.0000,0.000
2,78.8965,74.9640,70.4737,0.0000,0.000
3,80.7258,77.0671,72.8610,68.0865,0.000
4,82.4222,79.0295,75.1048,70.6171,65.556


In [9]:
def option_on_zcb(short_df, optype='c', opstyle='a', face_value=100, maturity=6, K=80, q=0.5):
    n=short_df.shape[0]-1
    n_zcb_df = zero_coupon_bond(short_df=short_df, face_value=100, maturity=n, q=q).iloc[:maturity+1, :maturity+1]
    tree = n_zcb_df.to_numpy()
    
    if optype == 'c':
        f=np.vectorize(lambda x:max(float(x)-float(K),0.0))
    else:
        f=np.vectorize(lambda x:max(float(K)-float(x),0.0))
        
    payoff = f(tree) 
    rate_tree = short_df.to_numpy()
    for i in range(maturity-1,-1,-1):
        for j in range(i+1):
            payoff[i][j]=max(payoff[i][j],1/(1+(rate_tree[i][j]/100))*(q*payoff[i+1][j+1]+(1-q)*payoff[i+1][j]))
    op_price_df = np.round(pd.DataFrame(data=payoff, columns=np.arange(payoff.shape[0])), 4)        
    return op_price_df 

option_on_zcb(short_df=short_df, optype='c', opstyle='e', face_value=100, maturity=6, K=80,q=0.5)

,0,1,2,3,4,5,6
0,2.3572,0.0000,0.0000,0.0000,0.0,0.0,0.0
1,3.3935,1.5567,0.0000,0.0000,0.0,0.0,0.0
2,4.6473,2.4451,0.8395,0.0000,0.0,0.0,0.0
3,6.0303,3.6408,1.4914,0.2891,0.0,0.0,0.0
4,7.4228,5.0774,2.5286,0.6166,0.0,0.0,0.0
5,8.7688,6.5639,3.9980,1.3071,0.0,0.0,0.0
6,10.0475,8.0079,5.5936,2.7551,0.0,0.0,0.0


In [10]:
r00 = 0.06
u = 1.1
d = 0.9
q = 1/2
n = 2
short_df = short_rate_tree(n=n, init_rate=r00, u=u, d=d)
short_df = short_df * 100
short_df

,0,1,2
0,6.00,0.00,0.00
1,5.40,6.60,0.00
2,4.86,5.94,7.26


In [13]:
zcb_2_df = zero_coupon_bond(short_df=short_df, face_value=100, maturity=n, q=0.5)
zcb_2_df

,0,1,2
0,89.002496,0.00000,0.0
1,94.876660,93.80863,0.0
2,100.000000,100.00000,100.0
